In [ ]:
# Imports principais
import pandas as pd
import numpy as np

# Modelos
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Métricas
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

# Lista global para armazenar os resultados de todas as rodadas
resultados_modelos = []

def treinar_e_avaliar(modelo, nome_modelo, parametros, x_train, y_train, x_val, y_val):
    """
    Treina e avalia um modelo com GridSearchCV e salva métricas para cada configuração testada.
    """
    gs = GridSearchCV(modelo, parametros, cv=3, scoring='r2', n_jobs=-1, return_train_score=True)
    gs.fit(x_train, y_train)
    
    # Armazena cada configuração rodada (não apenas o melhor)
    for i, params in enumerate(gs.cv_results_['params']):
        r2_train = gs.cv_results_['mean_train_score'][i]
        r2_val = gs.cv_results_['mean_test_score'][i]
        
        # Após GridSearch, avaliar com predição real na validação (para MAE e RMSE)
        modelo_temp = gs.estimator.set_params(**params)
        modelo_temp.fit(x_train, y_train)
        y_pred_val = modelo_temp.predict(x_val)

        mae_val = mean_absolute_error(y_val, y_pred_val)
        rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

        resultados_modelos.append({
            "Modelo": nome_modelo,
            "Configuração": params,
            "R2_Train": r2_train,
            "R2_Val": r2_val,
            "MAE_Val": mae_val,
            "RMSE_Val": rmse_val
        })

    return gs.best_estimator_, gs.best_params_
